## 네이버 영화평 감정분석

실제 영화 리뷰를 이용해 나이브 베이즈 분류기를 학습시키고, 입력 받은 영화평이 긍정 또는 부정적일 확률을 구하는 감정 분석(Sentiment Analysis)을 해보겠습니다. 데이터는 네이버 개발자 Lucy Park님의 Naver Sentiment Movie Corpus v1.0를 사용합니다. 네이버 영화의 140자 영화평을 모은 것으로 총 100,000개의 부정 리뷰, 100,000개의 긍정 리뷰로 구성되어 있습니다.
```
id	document	label
9251303	와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런게 진짜 영화지	1
10067386	안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.	1
2190435	사랑을 해본사람이라면 처음부터 끝까지 웃을수 있는영화	1
```

In [8]:
import io
import numpy
import matplotlib as mpl
mpl.use("Agg")
import matplotlib.pyplot as plt
import numpy as np
import elice_utils
import re
import math

special_chars_remover = re.compile("[^\w'|_]")
def remove_special_characters(sentence):
    return special_chars_remover.sub(' ', sentence)

def main():
    # read_data는 두 개의 string으로 이루어진 array를 리턴합니다.
    # array의 첫 번째 string은 모든 부정적 리뷰가 space로 나뉘어 합쳐진 string,
    # array의 두 번째 string은 모든 긍정적 리뷰가 space로 나뉘어 합쳐진 string입니다.
    training_sentences = read_data()



    testing_sentence = "어설픈 연기들로 몰입이 전혀 안되네요"
    
    prob_pair = naive_bayes(training_sentences, testing_sentence)
    # 시각화 코드입니다.
    plot_title = testing_sentence
    if len(plot_title) > 50: plot_title = plot_title[:50] + "..."
    visualize_boxplot(plot_title,
                  list(prob_pair),
                  ['Negative', 'Positive'])




def naive_bayes(training_sentences, testing_sentence):
    log_prob_negative = calculate_doc_prob(training_sentences[0], testing_sentence, 0.1) + math.log(0.5)
    log_prob_positive = calculate_doc_prob(training_sentences[1], testing_sentence, 0.1) + math.log(0.5)



    prob_pair = normalize_log_prob(log_prob_negative, log_prob_positive)
    
    return prob_pair




def read_data():



    training_sentences = [[], []]
    
    with open('ratings.txt') as fp:
        next(fp)
        for line in fp:
            splitted = line.split("\t")
            document = splitted[1]
            label = int(splitted[2])
            training_sentences[label].append(document)
    
    return [' '.join(training_sentences[0]), ' '.join(training_sentences[1])]




def normalize_log_prob(prob1, prob2):
    maxprob = max(prob1, prob2)

    prob1 -= maxprob
    prob2 -= maxprob
    prob1 = math.exp(prob1)
    prob2 = math.exp(prob2)

    normalize_constant = 1.0 / float(prob1 + prob2)
    prob1 *= normalize_constant
    prob2 *= normalize_constant

    return (prob1, prob2)

def calculate_doc_prob(training_sentence, testing_sentence, alpha):
    logprob = 0
    training_model = create_BOW(training_sentence)
    testing_model = create_BOW(testing_sentence)
    num_tokens = 0
    for w in training_model:
        num_tokens += training_model[w]
    for word in testing_model:
        cnt = testing_model[word]
        if (word in training_model):
            cnt_train = training_model[word]
            logprob += cnt * (math.log(cnt_train / num_tokens))
        else:
            logprob += cnt *(math.log(alpha / num_tokens))
    return logprob

def create_BOW(sentence):



    bow = {}
    
    # preprocessing
    sentence = sentence.lower()
    sentence = remove_special_characters(sentence)
    tokens = sentence.split()
    for token in tokens:
        if len(token) < 1:
            continue
        bow.setdefault(token, 0)
        bow[token] += 1




    return bow

def visualize_boxplot(title, values, labels):
    width = .35




    print(title)
    
    fig, ax = plt.subplots()
    ind = numpy.arange(len(values))
    rects = ax.bar(ind, values, width)
    ax.bar(ind, values, width=width)
    ax.set_xticks(ind + width/2)
    ax.set_xticklabels(labels)




    def autolabel(rects):
        # attach some text labels
        for rect in rects:
            height = rect.get_height()
            ax.text(rect.get_x()+rect.get_width()/2., height + 0.01, '%.2lf%%' % (height * 100), ha='center', va='bottom')

    autolabel(rects)

    plt.savefig("image.svg", format="svg")
    elice_utils.send_image("image.svg")

if __name__ == "__main__":
    main()

UnicodeDecodeError: 'cp949' codec can't decode byte 0xec in position 26: illegal multibyte sequence

In [7]:
data = pd.read_csv('ratings.txt', 'r')
data

C:\Users\csjty\AppData\Local\Temp\ipykernel_27092\495972172.py:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  data = pd.read_csv('ratings.txt', 'r')


ParserError: Error tokenizing data. C error: Expected 1 fields in line 131, saw 2


## Reference
- [Lucy Park님의 Naver Sentiment Movie Corpus v1.0](https://github.com/e9t/nsmc)